In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cirq
from cirq.contrib.svg import SVGCircuit
import random as rd
from sympy import *
import tensorflow as tf
import tensorflow_quantum as tfq
import math
import re
import itertools
from numpy import linalg as LA
from scipy.stats import poisson
import time
from random import choices
from random import uniform
import pandas as pd

2023-05-11 18:36:55.032549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 18:36:55.032563: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 18:36:56.246928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-11 18:36:56.246944: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-11 18:36:56.246954: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rafael-ThinkPad): /proc/driver/nvidia/version does not exist
2023-05-11 18:36:56.247096: I tensorflow/core/platform/cpu_fe

In [2]:
k = 5 #length of clauses
n_var = 7 #number of variables
nqubits = n_var #number of qubits in the circuit
p = 4 #number of layers for QAOA circuit
all_vars = [i for i in range(-n_var,n_var+1)]
all_vars = [i for i in all_vars if i != 0]

## Definitions

In [3]:
r_by_k = {2 : 1, 3: 6.43, 4: 20.43, 5 : 45.7, 6: 70.21, 7: 123.2, 8: 176.54, 10: 708.92, 16: 45425.2}

def generate_instance(k: int, n: int) -> np.ndarray:
    #generate an instance of random k-SAT with n variables in the satisfiability threshold
    if not (r := r_by_k.get(k)):
        raise ValueError(f"k must be in {list(r_by_k)} (got {k})")
    
    m = poisson(r*n).rvs()
    #return np.random.choice(all_vars, size=(m, k))
    all_variables = []
    all_signs = []
    for i in range(m):
        #all_signs.append([rd.choice(l) for i in range(k)])
        all_variables.append(choices(all_vars, k = k))

    all_variables = np.array(all_variables)
    #all_signs = np.array(all_signs)
    return all_variables

In [4]:
def generate_binary_strings(bit_count):
    binary_strings = []
    
    def genbin(n, bs=''):
        if len(bs) == n:
            binary_strings.append(bs)
        else:
            genbin(n, bs + '0')
            genbin(n, bs + '1')

    genbin(bit_count)
    return binary_strings

In [5]:
def dimacs_writer(dimacs_filename, cnf_array):
    #writes the dimacs file with the CNF
    cnf = cnf_array
    cnf_length = len(cnf)
    n_sat = len(cnf[0])
    var_num = np.max(cnf) 
    with open(dimacs_filename, "w") as f:

        f.write('c DIMACS file CNF '+str(n_sat)+'-SAT \n')
        f.write("p cnf {} {}\n".format(var_num, cnf_length))
        
        for i, clause in enumerate(cnf):
            line = clause.tolist()
            if i == cnf_length - 1:
                s = ' '.join(str(x) for x in line)+' 0'
                f.write(s)
            else: 
                s = ' '.join(str(x) for x in line)+' 0 \n'
                f.write(s)
                
class Verifier():
    #verifier from Qiskit page, takes a bit string and checks if cnf is satisfied
    def __init__(self, dimacs_file):
        with open(dimacs_file, 'r') as f:
            self.dimacs = f.read()

    def is_correct(self, guess):
        # Convert characters to bools & reverse
        guess = [bool(int(x)) for x in guess][::-1]
        for line in self.dimacs.split('\n'):
            line = line.strip(' 0')
            clause_eval = False
            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    # line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if clause_eval is False:
                return False
        return True

In [6]:
def my_gate(c, index):
    g = c * cirq.Z.on(qubits[index]) + cirq.I.on(qubits[index])
    return g

def ham_layer(diagonal, circuit, qubits, par):
    
    l = cirq.DiagonalGate(diagonal)._decompose_(qubits)
    l.pop(0)
    for j, gate in enumerate(l):

        if j % 2 == 0:
            dictn = gate._json_dict_()
            my_string = str(dictn['gate'])
            my_other_string = str(dictn['qubits'])
            number_p = re.findall("\d+\.\d+", my_string)
            res_p = [eval(i) for i in number_p]
            if '-' in my_string:
                sign = -1
            else:
                sign = 1
            
            number_q = re.findall(r'\d+', my_other_string)
            res_q = [eval(i) for i in number_q]
            kernel = sign*par*res_p[0]*np.pi
            rzgate = cirq.rz(kernel).on(qubits[res_q[1]])
            circuit.append(rzgate)
        else:
            circuit.append(gate)

def mixing_circuit(circuit, qubits, par):
    for i in range(len(qubits)):
        circuit.append(cirq.rx(par).on(qubits[i]))
    return circuit

In [7]:
def circuit_from_dimacs(dimacs_file, binary_strings, nqubits, qubits, layers, parameters):
    
    with open(dimacs_file, 'r') as f:
            dimacs = f.read()
    
    unsat_list = []

    for key in binary_strings:
        guess = [bool(int(x)) for x in key][::-1]

        clause_eval_list = []
        counter = 0
        for j, line in enumerate(dimacs.split('\n')):

            line = line.strip(' 0')
            clause_eval = False

            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    #line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if j > 1:
                counter += 1
                clause_eval_list.append(clause_eval)
        unsat_clauses = counter - sum(clause_eval_list)
        unsat_list.append(unsat_clauses)

    diagonal = unsat_list
    combinations = [p for p in itertools.product([1, -1], repeat=nqubits)]

    ops_list = []
    for j, combination in enumerate(combinations):
        ops_list.append((diagonal[j]/2**nqubits)*math.prod([my_gate(combination[i], i) for i in range(nqubits)]))

    cost = np.sum(ops_list)
    cost_m = cost.matrix()
    gs_energy = np.real(min(LA.eig(cost_m)[0]))
    
    qaoa_circuit = cirq.Circuit()
    num_param = 2 * layers

    for i in range(layers):
        ham_layer(diagonal, qaoa_circuit, qubits, parameters[2 * i])
        mixing_circuit(qaoa_circuit, qubits, parameters[2 * i + 1])

    return qaoa_circuit, cost

## Training

In [8]:
batches = 100
epochs = 1e4
parameter_list = []

for batch in range(batches):
    
    qubits = [cirq.GridQubit(0,i) for i in range(nqubits)]
    parameters = symbols("q0:%d" % (2*p))
    valid_keys = []
    dimacs_file = "data/random_cnf_BM_temp.dimacs" 
    binary_strings = generate_binary_strings(nqubits)
    
    while not valid_keys:
        #only accepts satisfiable CNFs
        inst = generate_instance(k, n_var)
        dimacs_writer(dimacs_file, inst)
        v = Verifier(dimacs_file)

        for key in binary_strings:
            if v.is_correct(key) == True:
                valid_keys.append(key)
    
    qaoa_circuit, cost = circuit_from_dimacs(dimacs_file, binary_strings, nqubits, qubits, p, parameters)
    cost_m = cost.matrix()
    gs_energy = np.real(min(LA.eig(cost_m)[0]))

    initial = cirq.Circuit()

    for qubit in qubits:
        initial.append(cirq.H(qubit)) #applying Hadamard to all qubits before running circuit
    
    inputs = tfq.convert_to_tensor([initial])
    ins = tf.keras.layers.Input(shape = (), dtype = tf.dtypes.string)
    outs = tfq.layers.PQC(qaoa_circuit, cost)(ins)
    ksat = tf.keras.models.Model(inputs = ins, outputs = outs)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)

    all_pars = [0.01*(-1)**(-i%2) for i in range(1,2*p+1)]
    ksat.trainable_variables[0].assign(all_pars)

    losses = []

    start = time.time()

    j=0
    
    while j < epochs:
        with tf.GradientTape() as tape:
            error = ksat(inputs)

        grads = tape.gradient(error, ksat.trainable_variables)
        opt.apply_gradients(zip(grads, ksat.trainable_variables))
        error = error.numpy()[0,0]
        losses.append(error)

        print('Batch is '+str(batch)+', valid keys are '+str(valid_keys)+' epoch is '+str(j)+' and absolute value of (ground state energy - error) is ' + str(abs(gs_energy - error)), end = '\r')

        j += 1
    
    params = ksat.get_weights()[0]
    end = time.time()
    parameter_list.append(params)

In [9]:
df = pd.DataFrame(parameter_list)
df

,0,1,2,3,4,5,6,7
0,0.139388,0.914613,0.216253,0.816341,0.315476,0.719058,0.406341,0.437478
1,0.127068,0.904259,0.204188,0.861888,0.250427,0.796092,0.313851,0.494623
2,0.188585,0.947678,0.292130,0.826406,0.344142,0.751837,0.386329,0.456121
3,0.158996,0.891575,0.251618,0.836794,0.345802,0.669185,0.487251,0.367255
4,0.201993,0.896003,0.321001,0.824482,0.384083,0.781588,0.421716,0.549679
...,...,...,...,...,...,...,...,...
95,0.136938,0.905853,0.205817,0.854245,0.276400,0.662132,0.331998,0.402924
96,0.169925,0.935527,0.271579,0.821127,0.338591,0.643931,0.394575,0.402615
97,0.158761,0.952558,0.238750,0.885676,0.317358,0.780440,0.487235,0.492969
98,0.150583,0.938087,0.220719,0.849646,0.275458,0.780142,0.385575,0.493464


In [10]:
df.mean(axis=0), df.var(axis=0)

(0    0.164485
 1    0.929594
 2    0.251417
 3    0.850440
 4    0.316355
 5    0.740633
 6    0.394623
 7    0.461315
 dtype: float32,
 0    0.000364
 1    0.000916
 2    0.000956
 3    0.002515
 4    0.001262
 5    0.006857
 6    0.001812
 7    0.004999
 dtype: float32)

In [11]:
df.to_csv('data/pars_k_'+str(k)+'_nvar_'+str(n_var)+'_layers_'+str(p)+'.csv')  